In [1]:
import turicreate as tc

In [2]:
prod = tc.SFrame('m_bfaa91c17752f745.frame_idx')

In [3]:
selected_words = ['awesome', 'great', 'fantastic', 'amazing', 'love', 'horrible', 'bad', 'terrible', 'awful', 'wow', 'hate']

In [4]:
prod['word_count'] = tc.text_analytics.count_words(prod['review'])

In [5]:
for i in range(len(selected_words)):
    def sel_count(word_count):
        if selected_words[i] in dict(word_count):
            return word_count[selected_words[i]]
        else:
            return 0
    prod[selected_words[i]] = prod['word_count'].apply(sel_count)

#Ans 1

In [6]:
for i in selected_words:
    print (i, prod[i].sum())

awesome 4075.0
great 59536.0
fantastic 1765.0
amazing 2726.0
love 43867.0
horrible 1245
bad 4950.0
terrible 1282
awful 753
wow 461
hate 1285


In [7]:
prod = prod[prod['rating'] != 3]

In [8]:
prod['sentiment'] = prod['rating'] >= 4

In [9]:
train_data,test_data = prod.random_split(.8, seed=0)

In [11]:
selected_words_model = tc.logistic_classifier.create(train_data,target='sentiment', features=selected_words, validation_set=test_data)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133448

Number of classes           : 2

Number of feature columns   : 11

...

| 4         | 5        | 1.550970     | 0.847708          | 0.846385            |

| 5         | 6        | 1.652994     | 0.847708          | 0.846385            |

| 6         | 7        | 1.755326     | 0.847708          | 0.846385            |

+-----------+----------+--------------+-------------------+---------------------+

SUCCESS: Optimal solution found.

In [12]:
coefficients = selected_words_model.coefficients

In [13]:
coef = coefficients.sort('value', ascending=False)

#Ans 2

In [14]:
print (coef.head())
print('***************************')
print (coef.tail())

+-------------+-------+-------+----------------------+----------------------+
|     name    | index | class |        value         |        stderr        |
+-------------+-------+-------+----------------------+----------------------+
|     love    |  None |   1   |  1.359268866922504   | 0.02806830015209953  |
| (intercept) |  None |   1   |  1.3365913848877726  | 0.008929969787655916 |
|   awesome   |  None |   1   |  1.133534666034134   | 0.08399643983187562  |
|   amazing   |  None |   1   |  1.100093311366018   |  0.0995477626046598  |
|  fantastic  |  None |   1   |   0.88580475688142   | 0.11167591293399651  |
|    great    |  None |   1   |  0.8630655001196439  | 0.018955052444376685 |
|     wow     |  None |   1   | -0.00953823606768843 | 0.16046411224711649  |
|     bad     |  None |   1   | -0.9914778800650664  | 0.03848428664699063  |
|     hate    |  None |   1   | -1.3484407222463202  | 0.07715698604297323  |
|    awful    |  None |   1   | -2.0529082040313598  | 0.1009973

#Ans 3

In [15]:
print ('Sentiment_Model Accuracy: 0.9176975738650012')
selected_words_model.evaluate(test_data)


Sentiment_Model Accuracy: 0.9176975738650012


{'accuracy': 0.8463848186404036,
 'auc': 0.6936022046674926,
 'confusion_matrix': Columns:
 	target_label	int
 	predicted_label	int
 	count	int
 
 Rows: 4
 
 Data:
 +--------------+-----------------+-------+
 | target_label | predicted_label | count |
 +--------------+-----------------+-------+
 |      1       |        0        |  159  |
 |      0       |        0        |  371  |
 |      0       |        1        |  4957 |
 |      1       |        1        | 27817 |
 +--------------+-----------------+-------+
 [4 rows x 3 columns],
 'f1_score': 0.9157860082304526,
 'log_loss': 0.39622654670876317,
 'precision': 0.8487520595594068,
 'recall': 0.9943165570488991,
 'roc_curve': Columns:
 	threshold	float
 	fpr	float
 	tpr	float
 	p	int
 	n	int
 
 Rows: 100001
 
 Data:
 +-----------+--------------------+-----+-------+------+
 | threshold |        fpr         | tpr |   p   |  n   |
 +-----------+--------------------+-----+-------+------+
 |    0.0    |        1.0         | 1.0 | 27976 | 53

In [16]:
prod['predicted_sentiment'] = selected_words_model.predict(prod, output_type = 'probability')

In [20]:
test_data['sentiment'].sum()/len(test_data['sentiment'])

0.8400192169108815

In [21]:
diaper_champ_reviews = prod[prod['name']=='Baby Trend Diaper Champ']

In [22]:
diaper_champ_reviews = diaper_champ_reviews.sort('predicted_sentiment', ascending=False)

In [26]:
diaper_champ_reviews

name,review,rating,word_count,awesome,great,fantastic
Baby Trend Diaper Champ,I LOVE LOVE LOVE thisproduct! It is SO much ...,4.0,"{'sorry': 1.0, 'be': 1.0,'will': 1.0, ...",0.0,1.0,0.0
Baby Trend Diaper Champ,I received my DiaperChamp at my baby shower ...,5.0,"{'second': 2.0, 'have':1.0, 'are': 1.0, 'pull': ...",0.0,0.0,0.0
Baby Trend Diaper Champ,"Love it, love it, loveit! This lives up to ...",5.0,"{'parents': 1.0, 'new':1.0, 'recommended': 1.0, ...",0.0,0.0,0.0
Baby Trend Diaper Champ,"Let me just say, I LOVETHIS PRODUCT!! I used ...",5.0,"{'friends': 1.0, 'feces':1.0, 'tank': 1.0, ...",0.0,1.0,1.0
Baby Trend Diaper Champ,I have been using thisdiaper pail for 41/2 ...,5.0,"{'recommend': 1.0,'would': 1.0, 'with': ...",0.0,2.0,0.0
Baby Trend Diaper Champ,Works great - no smells.LOVE that it uses reg ...,5.0,"{'plastic': 1.0, 'in':1.0, 'each': 1.0, ...",0.0,2.0,0.0
Baby Trend Diaper Champ,I love this diaper paleand wouldn't dream of ...,5.0,"{'easy': 1.0, 'how': 1.0,'like': 1.0, 'on': 1.0, ...",0.0,2.0,0.0
Baby Trend Diaper Champ,I love this diaper pail.It keeps the diapers ...,4.0,"{'wish': 1.0, 'is': 1.0,'stars': 1.0, 'gave': ...",0.0,0.0,0.0
Baby Trend Diaper Champ,I have a two-year-old sonand I love the Diaper ...,5.0,"{'close': 1.0, 'sticky':1.0, 'tightly': 1.0, ...",0.0,0.0,0.0
Baby Trend Diaper Champ,I love this diaper pail!It's so easy to use a ...,5.0,"{'some': 1.0, 'save':1.0, 'refills': 1.0, ...",0.0,0.0,0.0


#Ans 4

In [23]:
diaper_champ_reviews[0]['review']

'I LOVE LOVE LOVE this product! It is SO much easier to use than the Diaper Genie, (you need a PHD in poopy to figure out how to use the darn thing!) and it even takes the same bags as my kitchen trash can, shich is super convenient, and cost efficient as I can buy them in bulk.The only reason for not rating it a 5 star was that I did have one small problem with it. The foam gasket in the barrell which keeps the poopy smell inside the unit ripped somehow, and it got VERY stinky. HOWEVER, I contacted the manufacturer though their website, and received an email back the same day stating that this was unusual, and that replacement gaskets were on their way to me. They arrived inside of a week and after replacing, it works great again! (They even sent me extras should it happen again)I HIGHLY reccomend this diaper pail over ANY competitors, you will not be sorry!'

In [28]:
for i in range(len(diaper_champ_reviews)):
    if diaper_champ_reviews[i]['review'] == "I read a review below that can explain exactly what we experienced. We've had it for 16 months and it has worked wonderful for us. No smells, change it out once a week, easy to clean. Then a diaper snagged this foam material in the head part, so I pulled the rest of the foam out. Big mistake!!! Now it can no loner retain the stinkiness and we're looking for a replacement. Be careful of overloading and never take out that foam piece that is cushioned between pieces. I have figured out that it is key to keeping the stink out.":
        print (i)

250


In [29]:
diaper_champ_reviews[250]['predicted_sentiment']

0.7919288370624482